In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process #have to pip install into anaconda root

C:\Users\seans\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def movie_suggestion_by_favorite (title):
    df = pd.read_csv('movie_dataset.csv')
    features = ['overview','keywords','cast', 'genres','director']
    

    title = process.extractOne(title,df['title'])[0]  #uses fuzzy matching to extract the input to a movie closest to whats typed
    
    for feature in features:
        df[feature]=df[feature].fillna('') #filling any missing values with empty
        
    def combined_features(row):
        return row['overview']+' '+row['keywords']+' '+row['cast']+' '+row['genres']+' '+row['director']
   
    df['combined_features']=df.apply(combined_features, axis=1) #axis =1 means to do on all rows
    
    df['title'] = df['title'].apply(lambda x: x.title())
    
    count_matrix = CountVectorizer().fit_transform(df['combined_features'])
    
    cosine_sim = cosine_similarity(count_matrix)
    def get_title_from_index(index):
        return df[df.index==index]['title'].values[0]

    #get index from title of movie function
    def get_index_from_title(title):
        return df[df.title==title]['index'].values[0]
    
    movie_that_user_likes = title
    
    #checks to see if the inputed movie is in the database and if so looks up its index number
    if movie_that_user_likes in df['title'].values:
        movie_index = get_index_from_title(movie_that_user_likes)
        print(movie_that_user_likes,'is index',movie_index)
    else:
        print(movie_that_user_likes,'is not in the database. Try again!')

    similar_movies = list(enumerate(cosine_sim[movie_index]))
    
    sorted_similar_movies = sorted(similar_movies, key = lambda x:x[1], reverse=True)[1:]
    
    i=0
    print('The top 5 similar movies to '+ movie_that_user_likes +' are:')

    for element in sorted_similar_movies:
        print(get_title_from_index(element[0]))
        i = i + 1
        if i >=5:
            break


In [5]:
movie_suggestion_by_favorite('herculs')

Hercules is index 304
The top 5 similar movies to Hercules are:
The Legend Of Hercules
Gladiator
In The Heart Of The Sea
Jack The Giant Slayer
Star Trek Iv: The Voyage Home
